In [2]:
import requests
from pathlib import Path
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def fetch_input(day, year, save_dir="inputs"):
    """
    Fetches input for a specific Advent of Code day and saves it locally.

    :param day: The day of the Advent of Code (1-25).
    :param year: The Advent of Code year.
    :param save_dir: Directory to save the input file.
    :return: Path to the input file.
    """
    session_token = os.getenv("SESSION_TOKEN")
    if not session_token:
        raise ValueError("Session token not found in environment variables.")

    url = f"https://adventofcode.com/{year}/day/{day}/input"
    headers = {
        "Cookie": f"session={session_token}",
        "User-Agent": "https://github.com/sai-pendyala/Advent-of-Code-2024",
    }

    # Create the directory if it doesn't exist
    Path(save_dir).mkdir(exist_ok=True)
    input_file = Path(save_dir) / f"day{day:02}_input.txt"

    # Fetch input
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        with open(input_file, "w") as f:
            f.write(response.text.strip())
        print(f"Input for Day {day} saved to {input_file}")
        return input_file
    elif response.status_code == 404:
        print("Puzzle input is not available yet. Check after the problem is released.")
    else:
        print(f"Failed to fetch input: {response.status_code}")
    return None

In [3]:
def read_input(day, save_dir="inputs"):
    """
    Reads the content of the input file for a specific day of Advent of Code.

    :param day: The day of the Advent of Code (1-25).
    :param save_dir: Directory where the input file is saved (default is 'inputs').
    :return: Content of the input file as a string.
    """
    input_file = Path(save_dir) / f"day{day:02}_input.txt"

    if input_file.exists():
        with open(input_file, "r") as file:
            content = file.read().strip()  # Read and strip any surrounding whitespace
        print(f"Content of Day {day} input:\n{content}")
        return content
    else:
        print(f"Input file for Day {day} does not exist. Please fetch the input first.")
        return None

In [4]:
# Daily Workflow
from pathlib import Path
from datetime import datetime

# Dynamic Day and Year
now = datetime.utcnow()
DAY = now.day
YEAR = now.year

# Fetch input
fetch_input(DAY, YEAR)

# Read input
input_data = read_input(DAY)

/tmp/ipykernel_21995/4008538322.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()


Input for Day 9 saved to inputs/day09_input.txt
Content of Day 9 input:
7889985818705437406445235215557494675897197896477956638075433571904034142244995284823213398813689135711440962896562420828845124882727559461244587995344030583278125076902246579749311272513984382442219974906342354489109387361276162640381787907086827920398053362422897520233347317038617618585023854417993592782880133941691094119866977956618320419879131036875042412696195454461120673850833788955350632391194648884524134980648710336371846050793252449758484819695195305421824019604226696535775255525553289828124376205488678130935290466466568993229842353542674943734363218231225820196069403457252573697641808660317127876135907412662498672885627632407268412366503840449879527082715287143841692495224662237035978442662876483579404923497770383432259777573595434542456387125698566735601631818972149828866784755112872460815019256086682976996828371128349184171344484561361075538418387916176620978117431782233728975921807299134779701850418312

# Day 1

In [ ]:
lefts = []
rights = []
for line in input_data.split("\n"):
  left, right = line.strip().split()
  lefts.append(int(left))
  rights.append(int(right))

In [ ]:
def get_total_distance(lefts, rights):
  total_distance = 0
  for left, right in zip(lefts, rights):
    total_distance += abs(left - right)
  return total_distance

get_total_distance(sorted(lefts), sorted(rights))

In [ ]:
def get_similarity_score(lefts, rights):
  from collections import Counter
  similarity = 0
  right_counts = Counter(rights)
  for left in lefts:
    if left in right_counts:
      similarity += left * right_counts[left]
  return similarity

get_similarity_score(lefts, rights)

# Day 2

In [ ]:
def is_safe(report):
    sign = 1
    if report[1] < report[0]:
        sign = -1

    for i in range(1, len(report)):
        diff = report[i] - report[i - 1]
        if sign * diff < 0 or not 1 <= abs(diff) <= 3:
            return False  
    return True

In [ ]:
total_safe = 0
for line in input_data.split("\n"):
    check = is_safe(list(map(int, line.split())))
    total_safe += check

total_safe

In [ ]:
total_safe2 = 0
for line in input_data.split("\n"):
    report = list(map(int, line.split()))
    for i in range(len(report)):
        first_half = report[:i] if i > 0 else []
        second_half = report[i + 1:] if i < len(report) - 1 else []
        check = is_safe(first_half + second_half)
        if check:
            total_safe2 += 1
            break

total_safe2

# Day 3

In [ ]:
import re
pattern = r"mul\(\d+,\d+\)"
matches = re.findall(pattern, input_data)

total = 0
for match in matches:
    num1, num2 = map(int, match.strip("mul()").split(","))
    total += num1 * num2
total

In [ ]:
import re
pattern = r"mul\(\d+,\d+\)|do\(\)|don't\(\)"
matches = re.findall(pattern, input_data)

total = 0
do = 1
for match in matches:
    if match == "do()":
        do = 1
    elif match == "don't()":
        do = 0
    else:
        num1, num2 = map(int, match.strip("mul()").split(","))
        total += num1 * num2 * do
total

# Day 4

In [ ]:
grid = []
for line in input_data.split("\n"):
    grid.append(list(line))
m, n = len(grid), len(grid[0])

In [ ]:
def dfs(curr, x, y, dx, dy):
    if curr == "XMAS":
        return 1
    if not (0 <= x < m and 0 <= y < n):
        return 0
    if len(curr) > 4:
        return 0
    curr += grid[x][y]
    return dfs(curr, x + dx, y + dy, dx, dy)

In [ ]:
result = 0
for i in range(m):
    for j in range(n):
        if grid[i][j] == "X":
            for di, dj in [(-1, 0), (0, 1), (1, 0), (0, -1), (-1, 1), (1, 1), (1, -1), (-1, -1)]:
                result += dfs("", i, j, di, dj)

result

In [ ]:
result = 0
patterns = ["MAS", "SAM"]
for i in range(1, m - 1):
    for j in range(1, n - 1):
        if grid[i][j] == "A":
            diagonal1 = grid[i - 1][j - 1] + grid[i][j] + grid[i + 1][j + 1]
            diagonal2 = grid[i + 1][j - 1] + grid[i][j] + grid[i - 1][j + 1]
            # print(i, j, diagonal1, diagonal2)
            if diagonal1 in patterns and diagonal2 in patterns:
                result += 1
result

# Day 5

In [ ]:
def get_right_order(graph, degree):
    from collections import deque
    q = deque()
    for u, d in degree.items():
        if d == 0:
            q.append(u)
    right_order = []
    while q:
        node = q.popleft()
        right_order.append(node)
        for v in graph[node]:
            degree[v] -= 1
            if degree[v] == 0:
                q.append(v)
                
    return right_order

In [ ]:
from collections import defaultdict

rules = defaultdict(set)
updates = []

for line in input_data.split("\n"):
    if line == '':
        continue 
    if '|' in line:
        u, v = line.split('|')
        rules[u].add(v)
    else:
        updates.append(line.split(','))

print(rules, updates)

In [ ]:
result1 = 0
result2 = 0
for update in updates:
    degree = {u: 0 for u in update} 
    graph = defaultdict(set)

    for u in update:
        graph[u] = rules[u] & set(update)
        for v in graph[u]:
            degree[v] += 1
    right_order = get_right_order(graph, degree)
    if right_order == update:
        result1 += int(right_order[(len(update) - 1) // 2])
    else:
        result2 += int(right_order[(len(update) - 1) // 2])
    
result1, result2

# Day 6

In [ ]:
grid = []
for line in input_data.split("\n"):
    grid.append(list(line))

m, n = len(grid), len(grid[0])
start = None
directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]
for i in range(m):
    for j in range(n):
        if grid[i][j] == '^':
            start = (i, j)
            break
    if start is not None:
        break

In [ ]:
def travel(x, y, d_idx, path):
    while True:
        path.add((x, y))
        dx, dy = directions[d_idx]
        nx, ny = x + dx, y + dy 
        if not (0 <= nx < m and 0 <= ny < n):
            break 
        if grid[nx][ny] == '#':
            d_idx = (d_idx + 1) % 4
        else:
            x, y = nx, ny

In [ ]:
path = set()
travel(start[0], start[1], 0, path)
print(len(path))

In [ ]:
def simulate(x, y):
    vis = set()
    d_idx = 0
    while True:
        if (x, y, d_idx) in vis:
            return True
        vis.add((x, y, d_idx))

        dx, dy = directions[d_idx]
        nx, ny = x + dx, y + dy 

        if not (0 <= nx < m and 0 <= ny < n):
            break 

        if grid[nx][ny] == '#':
            d_idx = (d_idx + 1) % 4
        else:
            x, y = nx, ny
    
    return False

x, y = start
count = 0
for i, j in path:
    grid[i][j] = '#'
    count += simulate(x, y)
    grid[i][j] = '.'

count

# Day 7

In [16]:
tests = []

for line in input_data.split("\n"):
    test, nums = line.split(":")
    tests.append([ int(test), list(map(int, nums.split()))])

print(tests)

[[2382106471, [2, 8, 175, 1, 17, 3, 5, 9, 4, 51, 5]], [864708004, [278, 22, 259, 2, 12, 3]], [1659517050, [33, 8, 9, 673, 51, 967]], [5331437, [5, 3, 305, 89, 849]], [97828, [8, 6, 6, 3, 5, 70, 4, 696, 5, 91, 3]], [793977, [7, 93, 977]], [21316689, [8, 8, 3, 833, 67, 22, 19, 19]], [4685569, [3, 5, 3, 9, 9, 3, 4, 1, 746, 951, 7]], [4635637004, [7, 1, 9, 9, 87, 8, 14, 7, 8, 1, 78, 9]], [367804926120, [9, 900, 129, 43, 9, 8, 57, 40]], [3977374, [397, 7, 37, 4]], [28883667, [640, 6, 1, 9, 5, 2, 7, 59, 1, 6, 93]], [1219750, [7, 614, 28, 37, 5, 7, 2]], [54994, [7, 7, 5, 948, 46]], [192115489, [6, 3, 8, 47, 19, 81, 887, 4, 4, 8]], [434184384002, [916, 790, 61, 8, 22, 75, 8]], [263569049, [28, 6, 4, 881, 92]], [2598922, [4, 9, 3, 5, 511, 199, 94, 5, 58]], [2191959663, [42, 9, 796, 51, 63]], [11481664, [480, 8, 9, 3, 7, 21, 1, 85, 34]], [4043221, [8, 1, 7, 1, 21, 77, 10, 46, 8, 3]], [6479, [7, 3, 9, 3, 6, 15, 3, 7, 521, 7, 4]], [10772736, [480, 47, 5, 1, 2, 136, 8, 249]], [10349128, [4, 5, 58, 

In [17]:
def form_equations(i, nums, equation, equations):
  if i == len(nums):
    equation.pop()
    equations.add(tuple(equation[:]))
    equation = []
    return
  equation.append(nums[i])
  form_equations(i + 1, nums, equation + ['*'], equations)
  form_equations(i + 1, nums, equation + ['+'], equations)
  form_equations(i + 1, nums, equation + ['|'], equations)

In [18]:
def solve_equation(equation):
    result = equation[0]
    for i in range(1, len(equation), 2):
        operator = equation[i]
        operand = equation[i + 1]
        if operator == '*':
            result *= operand
        elif operator == '+':
            result += operand
        elif operator == '|':
            result = int(str(result) + str(operand))
    return result

In [19]:
result = 0
for test, nums in tests:
    equations = set()
    form_equations(0, nums, [], equations)
    for equation in equations:
        if solve_equation(equation) == test:
            result += test
            break
result

116094961956019

# Day 8

In [48]:
grid = []
for line in input_data.split('\n'):
    grid.append(list(line))

m = len(grid)
n = len(grid[0])

In [49]:
from collections import defaultdict

antennas = defaultdict(list)
for i in range(m):
    for j in range(n):
        if grid[i][j] != '.':
            antennas[grid[i][j]].append((i, j))

In [52]:
antinodes = set()

for signal, positions in antennas.items():
    if len(positions) > 1:
        for i in range(len(positions)):
            for j in range(i + 1, len(positions)):
                x1, y1 = positions[i]
                x2, y2 = positions[j]
                x3, y3 = x2 + (x2 - x1), y2 + (y2 - y1)
                x4, y4 = x1 - (x2 - x1), y1 - (y2 - y1)
                if 0 <= x3 < m and 0 <= y3 < n:
                    antinodes.add((x3, y3))
                if 0 <= x4 < m and 0 <= y4 < n:
                    antinodes.add((x4, y4))

len(antinodes)

293

In [ ]:
def extend_line(x1, y1, x2, y2):
    from math import gcd

    points = set()
    dx = x2 - x1
    dy = y2 - y1

    g = gcd(dx, dy)
    step_x = dx // g
    step_y = dy // g

    x, y = x2, y2
    while 0 <= x < m and 0 <= y < n:
        points.add((x, y))
        x += step_x
        y += step_y

    x, y = x1, y1
    while 0 <= x < m and 0 <= y < n:
        points.add((x, y))
        x -= step_x
        y -= step_y

    return points


In [54]:
antinodes = set()

for signal, positions in antennas.items():
    if len(positions) > 1:
        for i in range(len(positions)):
            for j in range(i + 1, len(positions)):
                x1, y1 = positions[i]
                x2, y2 = positions[j]
                line_points = extend_line(x1, y1, x2, y2)
                antinodes.update(line_points)

len(antinodes)

934

# Day 9

In [113]:
input_data = read_input(9)

Content of Day 9 input:
7889985818705437406445235215557494675897197896477956638075433571904034142244995284823213398813689135711440962896562420828845124882727559461244587995344030583278125076902246579749311272513984382442219974906342354489109387361276162640381787907086827920398053362422897520233347317038617618585023854417993592782880133941691094119866977956618320419879131036875042412696195454461120673850833788955350632391194648884524134980648710336371846050793252449758484819695195305421824019604226696535775255525553289828124376205488678130935290466466568993229842353542674943734363218231225820196069403457252573697641808660317127876135907412662498672885627632407268412366503840449879527082715287143841692495224662237035978442662876483579404923497770383432259777573595434542456387125698566735601631818972149828866784755112872460815019256086682976996828371128349184171344484561361075538418387916176620978117431782233728975921807299134779701850418312183149718270848574892499109814229029846519663594

In [114]:
disk_map = list(map(int, list(input_data)))

print(disk_map)

[7, 8, 8, 9, 9, 8, 5, 8, 1, 8, 7, 0, 5, 4, 3, 7, 4, 0, 6, 4, 4, 5, 2, 3, 5, 2, 1, 5, 5, 5, 7, 4, 9, 4, 6, 7, 5, 8, 9, 7, 1, 9, 7, 8, 9, 6, 4, 7, 7, 9, 5, 6, 6, 3, 8, 0, 7, 5, 4, 3, 3, 5, 7, 1, 9, 0, 4, 0, 3, 4, 1, 4, 2, 2, 4, 4, 9, 9, 5, 2, 8, 4, 8, 2, 3, 2, 1, 3, 3, 9, 8, 8, 1, 3, 6, 8, 9, 1, 3, 5, 7, 1, 1, 4, 4, 0, 9, 6, 2, 8, 9, 6, 5, 6, 2, 4, 2, 0, 8, 2, 8, 8, 4, 5, 1, 2, 4, 8, 8, 2, 7, 2, 7, 5, 5, 9, 4, 6, 1, 2, 4, 4, 5, 8, 7, 9, 9, 5, 3, 4, 4, 0, 3, 0, 5, 8, 3, 2, 7, 8, 1, 2, 5, 0, 7, 6, 9, 0, 2, 2, 4, 6, 5, 7, 9, 7, 4, 9, 3, 1, 1, 2, 7, 2, 5, 1, 3, 9, 8, 4, 3, 8, 2, 4, 4, 2, 2, 1, 9, 9, 7, 4, 9, 0, 6, 3, 4, 2, 3, 5, 4, 4, 8, 9, 1, 0, 9, 3, 8, 7, 3, 6, 1, 2, 7, 6, 1, 6, 2, 6, 4, 0, 3, 8, 1, 7, 8, 7, 9, 0, 7, 0, 8, 6, 8, 2, 7, 9, 2, 0, 3, 9, 8, 0, 5, 3, 3, 6, 2, 4, 2, 2, 8, 9, 7, 5, 2, 0, 2, 3, 3, 3, 4, 7, 3, 1, 7, 0, 3, 8, 6, 1, 7, 6, 1, 8, 5, 8, 5, 0, 2, 3, 8, 5, 4, 4, 1, 7, 9, 9, 3, 5, 9, 2, 7, 8, 2, 8, 8, 0, 1, 3, 3, 9, 4, 1, 6, 9, 1, 0, 9, 4, 1, 1, 9, 8, 6, 6, 9, 7, 7, 9, 5, 

In [115]:
block = []
id = 0
for i in range(0, len(disk_map), 2):
    file_block = disk_map[i] * [id]
    free_block = disk_map[i + 1] * ['.'] if i + 1 < len(disk_map) else []
    block.extend(file_block + free_block)
    id += 1

print(block)

[0, 0, 0, 0, 0, 0, 0, '.', '.', '.', '.', '.', '.', '.', '.', 1, 1, 1, 1, 1, 1, 1, 1, '.', '.', '.', '.', '.', '.', '.', '.', '.', 2, 2, 2, 2, 2, 2, 2, 2, 2, '.', '.', '.', '.', '.', '.', '.', '.', 3, 3, 3, 3, 3, '.', '.', '.', '.', '.', '.', '.', '.', 4, '.', '.', '.', '.', '.', '.', '.', '.', 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, '.', '.', '.', '.', 7, 7, 7, '.', '.', '.', '.', '.', '.', '.', 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, '.', '.', '.', '.', 10, 10, 10, 10, '.', '.', '.', '.', '.', 11, 11, '.', '.', '.', 12, 12, 12, 12, 12, '.', '.', 13, '.', '.', '.', '.', '.', 14, 14, 14, 14, 14, '.', '.', '.', '.', '.', 15, 15, 15, 15, 15, 15, 15, '.', '.', '.', '.', 16, 16, 16, 16, 16, 16, 16, 16, 16, '.', '.', '.', '.', 17, 17, 17, 17, 17, 17, '.', '.', '.', '.', '.', '.', '.', 18, 18, 18, 18, 18, '.', '.', '.', '.', '.', '.', '.', '.', 19, 19, 19, 19, 19, 19, 19, 19, 19, '.', '.', '.', '.', '.', '.', '.', 20, '.', '.', '.', '.', '.', '.', '.', '.', '.', 21, 21, 21, 21, 21, 21, 21, '.', '.', '.', 

In [71]:
i, j = 0, len(block) - 1
while i < j:
    # print(i, j)
    while i < j and block[i] != '.':
        i += 1
    while i < j and block[j] == '.':
        j -= 1
    block[i], block[j] = block[j], block[i]
    i += 1
    j += 1

print(block)

[0, 0, 9, 9, 8, 1, 1, 1, 8, 8, 8, 2, 7, 7, 7, 3, 3, 3, 6, 4, 4, 6, 5, 5, 5, 5, 6, 6, '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']


In [18]:
id = 0
checksum = 0
for x in block:
    if x == '.':
        break
    checksum += x * id
    id += 1
print(checksum)

1928


In [116]:
def find_free_spans(block, end_index):
    free_spans = []
    start = None
    for i in range(end_index):
        if block[i] == '.':
            if start is None:
                start = i
        else:
            if start is not None:
                free_spans.append((start, i - 1))
                start = None
    if start is not None:
        free_spans.append((start, end_index - 1))
    return free_spans

def move_files(block):
    max_id = max(b for b in block if b != '.')  
    for file_id in range(max_id, -1, -1): 
        file_indices = [i for i, b in enumerate(block) if b == file_id]
        if not file_indices:
            continue  
        file_start = file_indices[0]
        file_length = len(file_indices)
        # print(file_id, file_indices)
        free_spans = find_free_spans(block, file_start)
        for start, end in free_spans:
            if end - start + 1 >= file_length:
                block[start:start + file_length] = [file_id] * file_length
                for i in file_indices:
                    block[i] = '.'
                break
        # print(block)
    return block

print(move_files(block))

[0, 0, 0, 0, 0, 0, 0, 9998, 9998, 9998, 9998, 9997, 9997, 9997, 9985, 1, 1, 1, 1, 1, 1, 1, 1, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 9999, 2, 2, 2, 2, 2, 2, 2, 2, 2, 9996, 9996, 9996, 9996, 9996, 9996, 9981, 9978, 3, 3, 3, 3, 3, 9995, 9995, 9995, 9995, 9995, 9979, 9979, 9964, 4, 9994, 9994, 9994, 9994, 9994, 9994, 9994, 9962, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 9992, 9992, 9992, 9992, 7, 7, 7, 9987, 9987, 9987, 9987, 9968, 9968, 9968, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9984, 9984, 9984, 9984, 10, 10, 10, 10, 9982, 9982, 9982, 9982, 9946, 11, 11, 9967, 9967, 9941, 12, 12, 12, 12, 12, 9963, 9963, 13, 9977, 9977, 9977, 9977, 9922, 14, 14, 14, 14, 14, 9974, 9974, 9974, 9974, 9911, 15, 15, 15, 15, 15, 15, 15, 9972, 9972, 9972, 9972, 16, 16, 16, 16, 16, 16, 16, 16, 16, 9958, 9958, 9953, 9953, 17, 17, 17, 17, 17, 17, 9986, 9986, 9986, 9986, 9986, 9986, 9986, 18, 18, 18, 18, 18, 9993, 9993, 9993, 9993, 9993, 9993, 9993, 9993, 19, 19, 19, 19, 19, 19, 19, 19, 19, 9983, 9983, 9983, 9983, 9983

In [117]:
id = 0
checksum = 0
for x in block:
    if x != '.':
        checksum += id * x 
    id += 1

print(checksum)

6431472344710
